In [1]:
import pandas as pd
import numpy as np

# Load processed files
BASE = "../data/processed/"

df_apache = pd.read_csv(BASE + "apache_clean.csv")
df_ssh = pd.read_csv(BASE + "ssh_clean.csv")
df_unsw = pd.read_csv(BASE + "unsw_train_clean.csv")
df_merged = pd.read_csv(BASE + "final_merged.csv")

df_merged.head()

C:\Users\shash\AppData\Local\Temp\ipykernel_16804\3998753140.py:10: DtypeWarning: Columns (0,1,3,7,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv(BASE + "final_merged.csv")


,component,event_template,event_type,ip,is_login_attempt,is_success,label,level,pid,raw_message,source,timestamp,total_bytes,total_packets
0,NaN,workerEnv.init() ok <*>,E2,NaN,0.0,NaN,NaN,notice,NaN,workerEnv.init() ok /etc/httpd/conf/workers2.p...,apache,2005-12-04 04:47:44,NaN,NaN
1,NaN,mod_jk child workerEnv in error state <*>,E3,NaN,0.0,NaN,NaN,error,NaN,mod_jk child workerEnv in error state 6,apache,2005-12-04 04:47:44,NaN,NaN
2,NaN,jk2_init() Found child <*> in scoreboard slot <*>,E1,NaN,0.0,NaN,NaN,notice,NaN,jk2_init() Found child 6725 in scoreboard slot 10,apache,2005-12-04 04:51:08,NaN,NaN
3,NaN,jk2_init() Found child <*> in scoreboard slot <*>,E1,NaN,0.0,NaN,NaN,notice,NaN,jk2_init() Found child 6726 in scoreboard slot 8,apache,2005-12-04 04:51:09,NaN,NaN
4,NaN,jk2_init() Found child <*> in scoreboard slot <*>,E1,NaN,0.0,NaN,NaN,notice,NaN,jk2_init() Found child 6728 in scoreboard slot 6,apache,2005-12-04 04:51:09,NaN,NaN


In [2]:
df_merged['timestamp'] = pd.to_datetime(df_merged['timestamp'], errors='coerce')
df_ssh['timestamp'] = pd.to_datetime(df_ssh['timestamp'], errors='coerce')
df_apache['timestamp'] = pd.to_datetime(df_apache['timestamp'], errors='coerce')

In [3]:
df_merged['hour'] = df_merged['timestamp'].dt.hour
df_merged['day'] = df_merged['timestamp'].dt.day
df_merged['weekday'] = df_merged['timestamp'].dt.weekday
df_merged['minute'] = df_merged['timestamp'].dt.minute

In [4]:
df_ssh['failed_flag'] = (
    (df_ssh['is_login_attempt'] == 1) &
    (df_ssh['is_success'] == 0)
).astype(int)

df_ssh[['is_login_attempt', 'is_success', 'failed_flag']].head()

,is_login_attempt,is_success,failed_flag
0,0,0,0
1,1,0,1
2,1,0,1
3,0,0,0
4,0,0,0


In [5]:
ip_stats = df_ssh.groupby('ip').agg({
    'is_login_attempt': 'sum',
    'is_success': 'sum',
    'failed_flag': 'sum'
}).rename(columns={
    'is_login_attempt': 'total_attempts',
    'is_success': 'total_success',
    'failed_flag': 'total_failed'
})

ip_stats.head()

,total_attempts,total_success,total_failed
ip,,,
1.237.174.253,0,0,0
103.207.39.16,5,0,5
103.207.39.165,2,0,2
103.207.39.212,5,0,5
103.99.0.122,81,0,81


In [6]:
df_merged = df_merged.merge(ip_stats, on='ip', how='left')
df_merged.head(10)

,component,event_template,event_type,ip,is_login_attempt,is_success,label,level,pid,raw_message,...,timestamp,total_bytes,total_packets,hour,day,weekday,minute,total_attempts,total_success,total_failed
0,NaN,workerEnv.init() ok <*>,E2,NaN,0.0,NaN,NaN,notice,NaN,workerEnv.init() ok /etc/httpd/conf/workers2.p...,...,2005-12-04 04:47:44,NaN,NaN,4.0,4.0,6.0,47.0,NaN,NaN,NaN
1,NaN,mod_jk child workerEnv in error state <*>,E3,NaN,0.0,NaN,NaN,error,NaN,mod_jk child workerEnv in error state 6,...,2005-12-04 04:47:44,NaN,NaN,4.0,4.0,6.0,47.0,NaN,NaN,NaN
2,NaN,jk2_init() Found child <*> in scoreboard slot <*>,E1,NaN,0.0,NaN,NaN,notice,NaN,jk2_init() Found child 6725 in scoreboard slot 10,...,2005-12-04 04:51:08,NaN,NaN,4.0,4.0,6.0,51.0,NaN,NaN,NaN
3,NaN,jk2_init() Found child <*> in scoreboard slot <*>,E1,NaN,0.0,NaN,NaN,notice,NaN,jk2_init() Found child 6726 in scoreboard slot 8,...,2005-12-04 04:51:09,NaN,NaN,4.0,4.0,6.0,51.0,NaN,NaN,NaN
4,NaN,jk2_init() Found child <*> in scoreboard slot <*>,E1,NaN,0.0,NaN,NaN,notice,NaN,jk2_init() Found child 6728 in scoreboard slot 6,...,2005-12-04 04:51:09,NaN,NaN,4.0,4.0,6.0,51.0,NaN,NaN,NaN
5,NaN,workerEnv.init() ok <*>,E2,NaN,0.0,NaN,NaN,notice,NaN,workerEnv.init() ok /etc/httpd/conf/workers2.p...,...,2005-12-04 04:51:14,NaN,NaN,4.0,4.0,6.0,51.0,NaN,NaN,NaN
6,NaN,workerEnv.init() ok <*>,E2,NaN,0.0,NaN,NaN,notice,NaN,workerEnv.init() ok /etc/httpd/conf/workers2.p...,...,2005-12-04 04:51:14,NaN,NaN,4.0,4.0,6.0,51.0,NaN,NaN,NaN
7,NaN,workerEnv.init() ok <*>,E2,NaN,0.0,NaN,NaN,notice,NaN,workerEnv.init() ok /etc/httpd/conf/workers2.p...,...,2005-12-04 04:51:14,NaN,NaN,4.0,4.0,6.0,51.0,NaN,NaN,NaN
8,NaN,mod_jk child workerEnv in error state <*>,E3,NaN,0.0,NaN,NaN,error,NaN,mod_jk child workerEnv in error state 6,...,2005-12-04 04:51:18,NaN,NaN,4.0,4.0,6.0,51.0,NaN,NaN,NaN
9,NaN,mod_jk child workerEnv in error state <*>,E3,NaN,0.0,NaN,NaN,error,NaN,mod_jk child workerEnv in error state 6,...,2005-12-04 04:51:18,NaN,NaN,4.0,4.0,6.0,51.0,NaN,NaN,NaN


In [7]:
df_merged.fillna(0, inplace=True)

C:\Users\shash\AppData\Local\Temp\ipykernel_16804\3541803446.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df_merged.fillna(0, inplace=True)


In [8]:
df_ssh = df_ssh.sort_values('timestamp')

df_ssh['attempts_5min'] = (
    df_ssh
    .set_index('timestamp')['is_login_attempt']
    .rolling('5T')
    .sum()
    .reset_index(drop=True)
)

df_ssh[['timestamp','is_login_attempt','attempts_5min']].head(20)

C:\Users\shash\AppData\Local\Temp\ipykernel_16804\1662649736.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .rolling('5T')


,timestamp,is_login_attempt,attempts_5min
0,2025-12-10 06:55:46,0,0.0
1,2025-12-10 06:55:46,1,1.0
2,2025-12-10 06:55:46,1,2.0
3,2025-12-10 06:55:46,0,2.0
4,2025-12-10 06:55:46,0,2.0
5,2025-12-10 06:55:48,1,3.0
6,2025-12-10 06:55:48,0,3.0
7,2025-12-10 07:02:47,0,0.0
8,2025-12-10 07:07:38,1,1.0
9,2025-12-10 07:07:38,1,2.0


In [9]:
df_merged.to_csv(BASE + "final_featured.csv", index=False)
print("Saved: final_featured.csv")

Saved: final_featured.csv
